In [22]:
import specpy

from autosted.callback_buildingblocks.coordinate_value_wrappers import StageOffsetsSettingsGenerator
from autosted.callback_buildingblocks.regular_position_generators import PositionListOffsetGenerator
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover
from autosted.callback_buildingblocks.static_settings import FOVSettingsGenerator
from autosted.detection import SimpleFocusPlaneDetector
from autosted.callback_buildingblocks.value_wrappers import SimpleManualOffset
from autosted.callback_buildingblocks.data_selection import NewestDataSelector
from autosted.utils.tiling import centered_tiles
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.pipeline import AcquisitionPipeline
from autosted.imspector import get_current_stage_coords

In [31]:
# where to save & whether to save combined HDF5 file
save_folder = 'examples/acquisition_data/large_image_test'
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
# measurement_parameters = 'C:/Users/RESOLFT/Desktop/config_json/gabi/20240307_590_480_overview.json'
# alternative: use current from Imspector
measurement_parameters = specpy.get_application().value_at('', specpy.ValueTree.Measurement).get()

# yx FOV size
fov_size = [50e-6, 50e-6]

# yx number of tiles
n_tiles = [4, 4]

# how much the tiles should overlap (0-1)
overlap_fraction = 0.1

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

# generate regular grid around current stage position
# NOTE: we add empty z-fov size and 1 tile to get 3d coordinates
coordinate_list = centered_tiles(start_coords, fov_size=[0]+fov_size, n_tiles=[1]+n_tiles, overlap=overlap_fraction)
coordinate_list

In [33]:
# build pipeline object (just one level: 'field')
pipeline = AcquisitionPipeline(save_folder, ['field'], save_combined_hdf5=save_hdf5, name='multipoint-acquisition')

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('field',
    LocationRemover(JSONSettingsLoader(measurement_parameters)),
    PositionListOffsetGenerator(coordinate_list),
    FOVSettingsGenerator(lengths=[None] + fov_size),
    )

# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'field')

# start with initial task from callback
pipeline.run(next_position_generator)

In [ ]:
# channel to focus in
focus_channel = 0

# manual offset (zyx) to focus
manual_focus_offset = [0, 0, 0]

In [ ]:
# build pipeline object (just one level: 'field')
pipeline = AcquisitionPipeline(save_folder, ['field'], save_combined_hdf5=save_hdf5, name='multipoint-acquisition')

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('field',
    LocationRemover(JSONSettingsLoader(measurement_parameters)),
    PositionListOffsetGenerator(coordinate_list),
    FOVSettingsGenerator(lengths=[None] + fov_size),
    StageOffsetsSettingsGenerator(
        SimpleManualOffset(
            SimpleFocusPlaneDetector(
                NewestDataSelector(pipeline, level='field'),
                channel=focus_channel
        ), offset=manual_focus_offset))
    )

# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'field')

# start with initial task from callback
pipeline.run(next_position_generator)

In [34]:
from autosted.utils.parameter_constants import DIRECTION_STAGE, PIXEL_SIZE_PARAMETERS
from autosted.utils.dict_utils import get_parameter_value_array_from_dict
from calmutils.stitching import stitch
from calmutils.stitching.fusion import fuse_image
from calmutils.stitching.transform_helpers import translation_matrix

# index of flipped axes
flip_axes = [i for i,d in enumerate(DIRECTION_STAGE) if d < 0]

# generate dummy regular grid around current stage position with flipped coordinates
coordinate_list_for_stitch = centered_tiles(
    start_coords,
    fov_size=[0]+fov_size,
    n_tiles=[1]+n_tiles,
    overlap=overlap_fraction,
    flip_axes=flip_axes)

# get images of a particular channel and configuration
configuration = 0
channel = 0
images = [v.data[configuration][channel].squeeze() for v in pipeline.data.values()]

is2d = images[0].ndim == 2

# get pixel size
settings = pipeline.data[(0,)].measurement_settings[configuration]
pixel_sizes = get_parameter_value_array_from_dict(settings, PIXEL_SIZE_PARAMETERS)

# build (pixel-unit) transform matrix from coordinates
transforms = [translation_matrix((c/pixel_sizes)[(1 if is2d else 0):]) for c in coordinate_list_for_stitch]

# fuse into one image
fused = fuse_image(images, transforms)

In [ ]:
# alternative: with registration
transforms = stitch(images, [(c/pixel_sizes)[(1 if is2d else 0):] for c in coordinate_list_for_stitch], corr_thresh=0.9)

# fuse into one image
fused = fuse_image(images, transforms)

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(fused, cmap='magma')